In [1]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMClassifier
import os
import seaborn as sns
from wordcloud import WordCloud

In [2]:
df=pd.read_csv('./dataset/malicious_phish.csv')

print(df.shape)
df.head()

(651192, 2)


,url,type
0,https://www.google.com,benign
1,br-icloud.com.br,phishing
2,mp3raid.com/music/krizz_kaliko.html,benign
3,bopsecrets.org/rexroth/cr/1.htm,benign
4,http://www.garage-pirenne.be/index.php?option=...,defacement


In [3]:
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))

In [4]:
from urllib.parse import urlparse

def abnormal_url(url):
    # Extract the hostname from the URL using urlparse()
    hostname = urlparse(url).hostname
    if hostname is None:
        return 1
    
    # Split the hostname into subdomains
    subdomains = hostname.split('.')
    
    # Check if any subdomain appears twice in the hostname
    for i in range(len(subdomains)):
        if subdomains[i] in '.'.join(subdomains[i+1:]):
            return 1
    
    # Check if the extracted hostname matches the actual domain name of the website
    domain = '.'.join(subdomains[-2:])
    if domain not in url:
        return 1
    
    # If none of the checks above triggered a return statement, the URL is not abnormal
    return 0

df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))


In [5]:
df.head()

,url,type,use_of_ip,abnormal_url
0,https://www.google.com,benign,0,0
1,br-icloud.com.br,phishing,0,1
2,mp3raid.com/music/krizz_kaliko.html,benign,0,1
3,bopsecrets.org/rexroth/cr/1.htm,benign,0,1
4,http://www.garage-pirenne.be/index.php?option=...,defacement,0,0


In [6]:
from googlesearch import search

def google_index(url):
    try:
        # Perform a Google search for the URL
        results = search(url, num_results=5)
        # Check if the URL appears in the search results
        for result in results:
            if url in result:
                return 1
        return 0
    except Exception as e:
        # If an error occurs during the search, assume that the URL is not indexed
        return 0
df['google_index'] = df['url'].apply(lambda i: google_index(i))

In [7]:
df.head()

,url,type,use_of_ip,abnormal_url,google_index
0,https://www.google.com,benign,0,0,0
1,br-icloud.com.br,phishing,0,1,0
2,mp3raid.com/music/krizz_kaliko.html,benign,0,1,0
3,bopsecrets.org/rexroth/cr/1.htm,benign,0,1,0
4,http://www.garage-pirenne.be/index.php?option=...,defacement,0,0,0


In [8]:
df[df["type"]=="benign"].head(10)

,url,type,use_of_ip,abnormal_url,google_index
0,https://www.google.com,benign,0,0,0
2,mp3raid.com/music/krizz_kaliko.html,benign,0,1,0
3,bopsecrets.org/rexroth/cr/1.htm,benign,0,1,0
6,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,0,0,0
7,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,0,1,0
8,yourbittorrent.com/?q=anthony-hamilton-soulife,benign,0,1,0
10,allmusic.com/album/crazy-from-the-heat-r16990,benign,0,1,0
11,corporationwiki.com/Ohio/Columbus/frank-s-bens...,benign,0,1,0
13,myspace.com/video/vid/30602581,benign,0,1,0
17,quickfacts.census.gov/qfd/maps/iowa_map.html,benign,0,1,0


In [9]:
df.google_index.value_counts()

0    651192
Name: google_index, dtype: int64

In [10]:
df.type.value_counts()

benign        428104
defacement     96457
phishing       94111
malware        32520
Name: type, dtype: int64

In [11]:
def google_index(url):
    site = search(url, 5)
    return 1 if site else 0
df['google_index'] = df['url'].apply(lambda i: google_index(i))

In [12]:
df[df["type"]=="benign"].head(10)

,url,type,use_of_ip,abnormal_url,google_index
0,https://www.google.com,benign,0,0,1
2,mp3raid.com/music/krizz_kaliko.html,benign,0,1,1
3,bopsecrets.org/rexroth/cr/1.htm,benign,0,1,1
6,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,0,0,1
7,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,0,1,1
8,yourbittorrent.com/?q=anthony-hamilton-soulife,benign,0,1,1
10,allmusic.com/album/crazy-from-the-heat-r16990,benign,0,1,1
11,corporationwiki.com/Ohio/Columbus/frank-s-bens...,benign,0,1,1
13,myspace.com/video/vid/30602581,benign,0,1,1
17,quickfacts.census.gov/qfd/maps/iowa_map.html,benign,0,1,1


In [14]:
import requests
import re

def google_index(url):
    try:
        # Make a GET request to Google with the URL as the query
        response = requests.get(f"https://www.google.com/search?q={url}")
        # Search the response HTML for the URL and check if it appears
        match = re.search(rf'href=".+{url}"', response.text)
        if match:
            return 1
        else:
            return 0
    except Exception as e:
        # If an error occurs, assume the URL is not indexed
        return 0
df['google_index'] = df['url'].apply(lambda i: google_index(i))

KeyboardInterrupt: 

In [13]:
df.google_index.value_counts()

1    651192
Name: google_index, dtype: int64

In [18]:
def count_dot(url):
    count_dot = url.count('.')
    return count_dot

df['countdot'] = df['url'].apply(lambda i: count_dot(i))
df.head()

,url,type,use_of_ip,abnormal_url,google_index,count.,countdot
0,https://www.google.com,benign,0,0,1,2,2
1,br-icloud.com.br,phishing,0,1,1,2,2
2,mp3raid.com/music/krizz_kaliko.html,benign,0,1,1,2,2
3,bopsecrets.org/rexroth/cr/1.htm,benign,0,1,1,2,2
4,http://www.garage-pirenne.be/index.php?option=...,defacement,0,0,1,3,3


In [22]:
df[df["type"]=="phishing"].countdot.value_counts()

2     36198
3     26505
1     19388
4      6602
5      2110
6      1311
7       700
10      397
11      301
8       258
9       131
0        57
12       36
13       25
14       25
15       20
16       15
17        7
18        7
22        7
20        4
42        3
21        1
26        1
24        1
19        1
Name: countdot, dtype: int64